In [129]:
import os
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries
import csv
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score

# Loading historic stock data

In [130]:
key = 'E4M6BVAOLPUG2OL7'
ts = TimeSeries(key, output_format='pandas')

In [134]:
data, googl_meta_data = ts.get_daily('GOOGL', outputsize='full')
data = data.rename_axis(['Date'])
data

,1. open,2. high,3. low,4. close,5. volume
Date,,,,,
2019-11-27,1315.42,1317.64,1309.4742,1312.060,638435.0
2019-11-26,1309.91,1314.01,1304.0400,1313.000,940545.0
2019-11-25,1296.26,1310.41,1295.6100,1305.640,1179658.0
2019-11-22,1303.00,1306.90,1289.3600,1293.670,1388912.0
2019-11-21,1299.25,1311.12,1290.7500,1300.140,1277338.0
...,...,...,...,...,...
2004-08-25,104.76,108.00,103.8800,106.000,9188600.0
2004-08-24,111.24,111.60,103.5700,104.870,15247300.0
2004-08-23,110.76,113.48,109.0500,109.400,18256100.0


In [135]:
data = data.rename(columns={'1. open': 'Open', '2. high':'High', '3. low':'Low', '4. close':'Close', '5. volume':'Volume'})

In [136]:
path = str(os.getcwd()) + "/" + "data"
data.to_csv("trainData.csv")

# Feature Engineering

In [137]:
def read_train():
    prices = pd.read_csv("trainData.csv")
    prices = clean(prices)
    return prices

def clean(prices):
    (m, n) = prices.shape

    prices = prices.loc[:, ['Date', 'Open', 'Close']]

    # Add new column "Close Price at the Previous Day" and "Open Price at the Previous Day"
    prevColumn = prices.loc[:, ['Open', 'Close']].assign(index = lambda x: x.index-1)
    prevColumn = prevColumn.set_index('index')
    prices = prices.join(prevColumn, rsuffix='Prev')

    # Add new column "Close Price at the Day Before" and "Open Price at the Day Before"
    prevColumn = prices.loc[:, ['Open', 'Close']].assign(index = lambda x: x.index-2)
    prevColumn = prevColumn.set_index('index')
    prices = prices.join(prevColumn, rsuffix='Prev2')

    prevColumn = prices.loc[:, ['Open', 'Close']].assign(index = lambda x: x.index-3)
    prevColumn = prevColumn.set_index('index')
    prices = prices.join(prevColumn, rsuffix='Prev3')

    prevColumn = prices.loc[:, ['Open', 'Close']].assign(index = lambda x: x.index-4)
    prevColumn = prevColumn.set_index('index')
    prices = prices.join(prevColumn, rsuffix='Prev4')

    prevColumn = prices.loc[:, ['Open', 'Close']].assign(index = lambda x: x.index-5)
    prevColumn = prevColumn.set_index('index')
    prices = prices.join(prevColumn, rsuffix='Prev5')



    # No need now to save the fifth days data
    prices = prices[:m-5]

    return prices


def write_train(data):
    data.loc[:, ['Date','Open','OpenPrev','ClosePrev','OpenPrev2','ClosePrev2','OpenPrev3','ClosePrev3',
                 'OpenPrev4','ClosePrev4','OpenPrev5','ClosePrev5']].to_csv("trainX.csv", index=False)
    data.loc[:, ['Close']].to_csv( "trainY.csv",index=False)
    
def write_test(data):
    print(year_Y_2017)
    year_2017=data.loc[mask]
    year_2017.loc[:, ['Date','Open','OpenPrev','ClosePrev','OpenPrev2','ClosePrev2','OpenPrev3','ClosePrev3',
                 'OpenPrev4','ClosePrev4','OpenPrev5','ClosePrev5']].to_csv("trainX.csv", index=False)
    year_2017.loc[:, ['Close']].to_csv( "trainY.csv",index=False)

# Evaluate Linear Regression and predict stock price

In [186]:
def read_val():
    X = pd.read_csv("trainX.csv", header=0)
    y = pd.read_csv("trainY.csv", header=0)
    X['Date'] = pd.to_datetime(X['Date'], format='%Y-%m-%d')
    return X, y



def plot_trend(x, predict, y, title):
    plt.xticks(rotation=45)
    plt.plot_date(x, y, fmt='b-', xdate=True, ydate=False, label='Real value')
    plt.plot_date(x, predict, fmt='r-', xdate=True, ydate=False, label='Predicted value')
    plt.legend(loc='upper center')
    plt.ylabel('Close prices')
    plt.title(title)
    plt.grid()
    plt.show()


def modify(X, columns):
    columns.append('Date2num')
    returnX = X.loc[:, columns]
    return returnX


def predict_and_plot(X, date):
    m, n = X.shape
    X_test = X.loc[np.floor(m*0.6)+1:]
    X_train = X.loc[:np.floor(m*0.6)]
    
    y_test = y.loc[np.floor(m*0.6)+1:]
    y_train = y.loc[:np.floor(m*0.6)]
    
    date_test = date.loc[np.floor(m*0.6)+1:]
    date_train = date.loc[:np.floor(m*0.6)]
    
    lr = LinearRegression()

    lr.fit(X_train, y_train)

    print('Coefficients: \n', lr.coef_)
    print('Intercept: \n', lr.intercept_)
    pred = lr.predict(X_test)
    
    return lr,pred

In [198]:
prices_train=read_train()
write_train(prices_train)

X, y = read_val()

X['Date2num'] = X['Date'].apply(lambda x: mdates.date2num(x))
date = X.loc[:, ['Date']]
del X['Date']

X1 = modify(X, [])
predict_and_plot(X1,date)

Coefficients: 
 [[0.17890297]]
Intercept: 
 [-130806.5204398]


(LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
 array([[516.30979642],
        [516.13089345],
        [515.95199048],
        ...,
        [118.78739822],
        [118.25068931],
        [118.07178634]]))

# Predict stock price after dimensionality reduction

In [199]:
X3 = modify(X, ['Open','OpenPrev','ClosePrev'])
lr,close_val=predict_and_plot(X3, date)
X3

Coefficients: 
 [[ 1.00056646e+00  3.60356692e-02 -3.83317501e-02  5.08682983e-04]]
Intercept: 
 [-372.91109733]


,Open,OpenPrev,ClosePrev,Date2num
0,1315.42,1309.91,1313.00,737390.0
1,1309.91,1296.26,1305.64,737389.0
2,1296.26,1303.00,1293.67,737388.0
3,1303.00,1299.25,1300.14,737385.0
4,1299.25,1311.36,1301.86,737384.0
...,...,...,...,...
3837,102.70,102.32,102.37,731825.0
3838,102.32,105.28,102.01,731824.0
3839,105.28,108.10,106.15,731823.0
3840,108.10,104.95,107.91,731820.0


# Load the Business days only

In [189]:
from collections import OrderedDict
from datetime import datetime

daterange = pd.date_range('1995-01-01', datetime.today().strftime('%Y-%m-%d'), freq='B')
newDate = pd.DataFrame(data=daterange, columns=['date'])
newDate = newDate.dropna()
newDate

,date
0,1995-01-02
1,1995-01-03
2,1995-01-04
3,1995-01-05
4,1995-01-06
...,...
6493,2019-11-21
6494,2019-11-22
6495,2019-11-25
6496,2019-11-26


# Load the stock prices for the above dates and predict

In [190]:
prevOpen = data.loc[:,['Open']]
prevOpen = prevOpen.iloc[len(prevOpen)-1]
prevClose = close_val.item(len(close_val)-1)
prevOpen = prevOpen.values.item(0)
newX3 = X3
try:
    newX3.drop('Date2num',axis=1,inplace=True)
except:
    print('already dropped')
dummy = pd.DataFrame([[prevClose,prevOpen,prevClose]], columns=['Open','OpenPrev','ClosePrev'])
newX3 = newX3.append(dummy, ignore_index=True)
date = date.append(newDate, ignore_index=True)
for x in range(0,len(newDate)-1):
    print(len(newX3))
    print(len(date))
    lr,close_val = predict_and_plot(newX3, date)
    prevClose = close_val.item(len(close_val)-1)
    prevOpen = newX3.loc[:,['Open']]
    prevOpen = prevOpen.iloc[len(prevOpen)-1]
    prevOpen = prevOpen.values.item(0)
    dummy = pd.DataFrame([[prevClose,prevOpen,prevClose]], columns=['Open','OpenPrev','ClosePrev'])
    newX3 = newX3.append(dummy, ignore_index=True)

newX3

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


3843
10340
Coefficients: 
 [[ 1.0012921   0.03647955 -0.03792819]]
Intercept: 
 [0.05362603]
3844
10340
Coefficients: 
 [[ 1.00128836  0.03648502 -0.03792219]]
Intercept: 
 [0.04587441]
3845
10340
Coefficients: 
 [[ 1.00128847  0.03647566 -0.03791212]]
Intercept: 
 [0.0450569]
3846
10340
Coefficients: 
 [[ 1.00128847  0.03647566 -0.03791212]]
Intercept: 
 [0.0450569]
3847
10340
Coefficients: 
 [[ 1.00126027  0.03653377 -0.0379278 ]]
Intercept: 
 [0.03076804]
3848
10340
Coefficients: 
 [[ 1.00126027  0.03653377 -0.0379278 ]]
Intercept: 
 [0.03076804]
3849
10340
Coefficients: 
 [[ 1.0012621   0.03648432 -0.03788441]]
Intercept: 
 [0.03499483]
3850
10340
Coefficients: 
 [[ 1.0013501   0.03650091 -0.03800215]]
Intercept: 
 [0.04806583]
3851
10340
Coefficients: 
 [[ 1.0013501   0.03650091 -0.03800215]]
Intercept: 
 [0.04806583]
3852
10340
Coefficients: 
 [[ 1.00135537  0.03651739 -0.03802151]]
Intercept: 
 [0.04569445]
3853
10340
Coefficients: 
 [[ 1.00135537  0.03651739 -0.03802151]]
Inter

3941
10340
Coefficients: 
 [[ 1.00166132  0.03602758 -0.03797435]]
Intercept: 
 [0.17874281]
3942
10340
Coefficients: 
 [[ 1.00167618  0.03584013 -0.0378112 ]]
Intercept: 
 [0.18778977]
3943
10340
Coefficients: 
 [[ 1.00167618  0.03584013 -0.0378112 ]]
Intercept: 
 [0.18778977]
3944
10340
Coefficients: 
 [[ 1.00170961  0.03622172 -0.03825827]]
Intercept: 
 [0.21849291]
3945
10340
Coefficients: 
 [[ 1.00159424  0.03605364 -0.03795213]]
Intercept: 
 [0.19678804]
3946
10340
Coefficients: 
 [[ 1.00159424  0.03605364 -0.03795213]]
Intercept: 
 [0.19678804]
3947
10340
Coefficients: 
 [[ 1.00158066  0.03592814 -0.03779832]]
Intercept: 
 [0.1826646]
3948
10340
Coefficients: 
 [[ 1.00158066  0.03592814 -0.03779832]]
Intercept: 
 [0.1826646]
3949
10340
Coefficients: 
 [[ 1.00158764  0.03566607 -0.03752652]]
Intercept: 
 [0.16657831]
3950
10340
Coefficients: 
 [[ 1.0015912   0.03526664 -0.03710348]]
Intercept: 
 [0.14031115]
3951
10340
Coefficients: 
 [[ 1.0015912   0.03526664 -0.03710348]]
Inter

Coefficients: 
 [[ 1.00295427  0.0366928  -0.03962688]]
Intercept: 
 [-0.12471441]
4044
10340
Coefficients: 
 [[ 1.00295836  0.03672392 -0.0396713 ]]
Intercept: 
 [-0.11512402]
4045
10340
Coefficients: 
 [[ 1.00295967  0.03675506 -0.03970111]]
Intercept: 
 [-0.11785709]
4046
10340
Coefficients: 
 [[ 1.00295967  0.03675506 -0.03970111]]
Intercept: 
 [-0.11785709]
4047
10340
Coefficients: 
 [[ 1.00295105  0.03681465 -0.03975914]]
Intercept: 
 [-0.11050957]
4048
10340
Coefficients: 
 [[ 1.00295105  0.03681465 -0.03975914]]
Intercept: 
 [-0.11050957]
4049
10340
Coefficients: 
 [[ 1.0029647   0.03680254 -0.03975444]]
Intercept: 
 [-0.11702704]
4050
10340
Coefficients: 
 [[ 1.00297042  0.03680554 -0.03976128]]
Intercept: 
 [-0.11899718]
4051
10340
Coefficients: 
 [[ 1.00297042  0.03680554 -0.03976128]]
Intercept: 
 [-0.11899718]
4052
10340
Coefficients: 
 [[ 1.00297077  0.03680665 -0.03976294]]
Intercept: 
 [-0.11878777]
4053
10340
Coefficients: 
 [[ 1.00297077  0.03680665 -0.03976294]]
Inte

Coefficients: 
 [[ 1.00297061  0.03673538 -0.03967385]]
Intercept: 
 [-0.1396359]
4151
10340
Coefficients: 
 [[ 1.00297061  0.03673538 -0.03967385]]
Intercept: 
 [-0.1396359]
4152
10340
Coefficients: 
 [[ 1.00292241  0.03652003 -0.0393895 ]]
Intercept: 
 [-0.16314924]
4153
10340
Coefficients: 
 [[ 1.00292241  0.03652003 -0.0393895 ]]
Intercept: 
 [-0.16314924]
4154
10340
Coefficients: 
 [[ 1.00292397  0.03652261 -0.03938999]]
Intercept: 
 [-0.16727158]
4155
10340
Coefficients: 
 [[ 1.00292035  0.03651754 -0.03938099]]
Intercept: 
 [-0.16762003]
4156
10340
Coefficients: 
 [[ 1.00292035  0.03651754 -0.03938099]]
Intercept: 
 [-0.16762003]
4157
10340
Coefficients: 
 [[ 1.00290831  0.0366077  -0.03945355]]
Intercept: 
 [-0.1738444]
4158
10340
Coefficients: 
 [[ 1.00290831  0.0366077  -0.03945355]]
Intercept: 
 [-0.1738444]
4159
10340
Coefficients: 
 [[ 1.00289514  0.03669703 -0.03953474]]
Intercept: 
 [-0.16819094]
4160
10340
Coefficients: 
 [[ 1.00288219  0.03673294 -0.03955139]]
Intercep

...

Coefficients: 
 [[ 0.99944321  0.04255098 -0.04196319]]
Intercept: 
 [-0.21291126]
6008
10340
Coefficients: 
 [[ 0.99944321  0.04255098 -0.04196319]]
Intercept: 
 [-0.21291126]
6009
10340
Coefficients: 
 [[ 0.999444    0.04255572 -0.04196843]]
Intercept: 
 [-0.21315916]
6010
10340
Coefficients: 
 [[ 0.99942812  0.04245622 -0.04186625]]
Intercept: 
 [-0.20185397]
6011
10340
Coefficients: 
 [[ 0.99942812  0.04245622 -0.04186625]]
Intercept: 
 [-0.20185397]
6012
10340
Coefficients: 
 [[ 0.99942727  0.04249453 -0.04190889]]
Intercept: 
 [-0.19743104]
6013
10340
Coefficients: 
 [[ 0.99942727  0.04249453 -0.04190889]]
Intercept: 
 [-0.19743104]
6014
10340
Coefficients: 
 [[ 0.99944637  0.04245157 -0.04187794]]
Intercept: 
 [-0.20350435]
6015
10340
Coefficients: 
 [[ 0.99945088  0.04245    -0.04187515]]
Intercept: 
 [-0.20843007]
6016
10340
Coefficients: 
 [[ 0.99945088  0.04245    -0.04187515]]
Intercept: 
 [-0.20843007]
6017
10340
Coefficients: 
 [[ 0.99945011  0.04245322 -0.04187668]]
Inte

Coefficients: 
 [[ 0.99931322  0.04205463 -0.04144928]]
Intercept: 
 [-0.11865955]
6124
10340
Coefficients: 
 [[ 0.99932276  0.04207768 -0.04147986]]
Intercept: 
 [-0.12033908]
6125
10340
Coefficients: 
 [[ 0.99933549  0.04219472 -0.04161969]]
Intercept: 
 [-0.11200822]
6126
10340
Coefficients: 
 [[ 0.99933549  0.04219472 -0.04161969]]
Intercept: 
 [-0.11200822]
6127
10340
Coefficients: 
 [[ 0.99910749  0.04230129 -0.04148567]]
Intercept: 
 [-0.12232081]
6128
10340
Coefficients: 
 [[ 0.99910749  0.04230129 -0.04148567]]
Intercept: 
 [-0.12232081]
6129
10340
Coefficients: 
 [[ 0.99911888  0.04229849 -0.04150273]]
Intercept: 
 [-0.11536037]
6130
10340
Coefficients: 
 [[ 0.99911582  0.04230073 -0.04150142]]
Intercept: 
 [-0.11576131]
6131
10340
Coefficients: 
 [[ 0.99911582  0.04230073 -0.04150142]]
Intercept: 
 [-0.11576131]
6132
10340
Coefficients: 
 [[ 0.99912154  0.04227756 -0.04148847]]
Intercept: 
 [-0.11207615]
6133
10340
Coefficients: 
 [[ 0.99912154  0.04227756 -0.04148847]]
Inte

Coefficients: 
 [[ 0.9985406   0.0423395  -0.04093972]]
Intercept: 
 [-0.1364534]
6220
10340
Coefficients: 
 [[ 0.99855072  0.04235886 -0.04096514]]
Intercept: 
 [-0.13980204]
6221
10340
Coefficients: 
 [[ 0.99855072  0.04235886 -0.04096514]]
Intercept: 
 [-0.13980204]
6222
10340
Coefficients: 
 [[ 0.99856038  0.04235299 -0.0409727 ]]
Intercept: 
 [-0.13669728]
6223
10340
Coefficients: 
 [[ 0.99856038  0.04235299 -0.0409727 ]]
Intercept: 
 [-0.13669728]
6224
10340
Coefficients: 
 [[ 0.99856106  0.042353   -0.04097401]]
Intercept: 
 [-0.13620354]
6225
10340
Coefficients: 
 [[ 0.99856143  0.04235686 -0.04097803]]
Intercept: 
 [-0.13637197]
6226
10340
Coefficients: 
 [[ 0.99856143  0.04235686 -0.04097803]]
Intercept: 
 [-0.13637197]
6227
10340
Coefficients: 
 [[ 0.99858539  0.04249972 -0.04116303]]
Intercept: 
 [-0.12153041]
6228
10340
Coefficients: 
 [[ 0.99858539  0.04249972 -0.04116303]]
Intercept: 
 [-0.12153041]
6229
10340
Coefficients: 
 [[ 0.99857896  0.04237389 -0.04102122]]
Inter

 [[ 0.99877125  0.04278654 -0.04160866]]
Intercept: 
 [-0.14403617]
6322
10340
Coefficients: 
 [[ 0.99873987  0.04271776 -0.04149765]]
Intercept: 
 [-0.15293391]
6323
10340
Coefficients: 
 [[ 0.99873987  0.04271776 -0.04149765]]
Intercept: 
 [-0.15293391]
6324
10340
Coefficients: 
 [[ 0.99872789  0.04275468 -0.04151658]]
Intercept: 
 [-0.1578793]
6325
10340
Coefficients: 
 [[ 0.99873741  0.04281269 -0.04158799]]
Intercept: 
 [-0.15469119]
6326
10340
Coefficients: 
 [[ 0.99873741  0.04281269 -0.04158799]]
Intercept: 
 [-0.15469119]
6327
10340
Coefficients: 
 [[ 0.99866887  0.04299673 -0.04169035]]
Intercept: 
 [-0.16547556]
6328
10340
Coefficients: 
 [[ 0.99866887  0.04299673 -0.04169035]]
Intercept: 
 [-0.16547556]
6329
10340
Coefficients: 
 [[ 0.9986636   0.04290294 -0.04160327]]
Intercept: 
 [-0.15565448]
6330
10340
Coefficients: 
 [[ 0.9986624   0.04294735 -0.04165231]]
Intercept: 
 [-0.15089199]
6331
10340
Coefficients: 
 [[ 0.9986624   0.04294735 -0.04165231]]
Intercept: 
 [-0.150

ValueError: Found input variables with inconsistent numbers of samples: [3843, 3842]